## 순환 신경망

In [1]:
# 순환 신경망: 시퀸스 단위로 처리하는 스퀀스 모델
# 입력과 출력의 길이를 다르게 설계할 수 있음.
# RNN 셀의 각 입, 출력단위는 주로 단어 벡터

In [3]:
from tensorflow.keras.layers import SimpleRNN

model = SimpleRNN()
model.add(SimpleRNN(hidden_units))


TypeError: SimpleRNN.__init__() missing 1 required positional argument: 'units'

In [ ]:
# RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력
#! 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴합니다. 하지만, 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴합니다.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2, 10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 2D 텐서
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10)))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 3D 텐서
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10), return_sequences=True))
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 파이썬으로 RNN구현하기
import numpy as np

timesteps = 10
input_dim = 4
hidden_units = 8

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units,)) 

print('초기 은닉 상태 :',hidden_state_t)

초기 은닉 상태 : [0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
Wx = np.random.random((hidden_units, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_units, hidden_units)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_units,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

print('가중치 Wx의 크기(shape) :',np.shape(Wx))
print('가중치 Wh의 크기(shape) :',np.shape(Wh))
print('편향의 크기(shape) :',np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(shape) : (8,)


In [9]:
total_hidden_states = []

# 각 시점 별 입력값.
for input_t in inputs:

  # Wx * Xt + Wh * Ht-1 + b(bias)
  output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)

  # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
  # 각 시점의 은닉 상태의 값을 계속해서 누적
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해주는 용도.
total_hidden_states = np.stack(total_hidden_states, axis=0)

# (timesteps, output_dim)
print('모든 시점의 은닉 상태 :')
print(total_hidden_states)


모든 시점의 은닉 상태 :
[[0.87504622 0.51508439 0.41045536 0.79560342 0.82928792 0.91611041
  0.80202252 0.80237324]
 [0.99995244 0.99921677 0.99960384 0.99498194 0.99803617 0.99987055
  0.99936662 0.99930641]
 [0.99999841 0.99995077 0.99994523 0.99899945 0.99956907 0.99998127
  0.99996825 0.99992396]
 [0.99999842 0.99997221 0.99996074 0.9988597  0.99963746 0.99998593
  0.9999817  0.99994916]
 [0.99999842 0.99993973 0.99993489 0.99933073 0.99961959 0.99998269
  0.99995306 0.99990725]
 [0.99999773 0.99996971 0.99994069 0.99919838 0.99968058 0.99997238
  0.99994635 0.99989729]
 [0.99999938 0.99998675 0.99997047 0.99967983 0.99991442 0.99999302
  0.99998264 0.99996363]
 [0.99999911 0.99997191 0.99994431 0.99957514 0.99985539 0.99997365
  0.99994006 0.99990079]
 [0.99999644 0.99995556 0.99993289 0.99875773 0.9993927  0.99997079
  0.99994791 0.99988885]
 [0.99999937 0.99996834 0.99996569 0.99932391 0.99978992 0.99999126
  0.99998592 0.99996149]]


In [10]:
# 깊은 순환 신경망: 은닉층이 2개 이상인 신경망

In [ ]:
# 양방향 순환 신경망: Forward State, Backward State

## 장단기 메모리(LSTM)

In [ ]:
# 바닐라 RNN
